In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import nltk
import string
import re
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from spellchecker import SpellChecker
from nltk import FreqDist
from collections import Counter
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer(strip_handles=True)



In [6]:
#stop_words = set(stopwords.words('english')) {en llaves}
stop_words = stopwords.words('english') # [en corchetes]
#stop_words += ['im', 'amp', 'go', 'get', 'one','2','would','dont'] #otras palabras de propia eleccion

In [7]:
# Thanks to https://www.kaggle.com/rftexas/text-only-kfold-bert
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [8]:
df = pd.read_csv("dataset/trainSpellChecker.csv")
pd.set_option('display.max_colwidth', -1) #elimino limite de truncado para visualizacion
df.info()
df.loc[5315:5320]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7612 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


/home/pablo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,id,keyword,location,text,target
5315,7589,outbreak,?? ?+254? ? \??å¡_??å¡_???å¡_?/??,families to sue over legionnaires more than families affected by the fatal outbreak of legionnaires dista,1
5316,7590,outbreak,NaN,an outbreak of legionnaires disease in new york has killed at least people ûó now officials think they fo,1
5317,7591,outbreak,Dammam- KSA,families to sue over legionnaires more than families affected by the fatal outbreak of legionnaires dista,1
5318,7593,outbreak,"New York, NY",an outbreak of legionnaires disease in new york has killed at least people,1
5319,7594,outbreak,Pro-American and Anti-#Occupy,death confirmed in legionnaires outbreak in south bronx total cases reaches triple digits,1
5320,7595,pandemonium,NaN,tube strike absolute pandemonium,0


In [9]:
df_test= pd.read_csv("dataset/test.csv")
pd.set_option('display.max_colwidth', -1) #elimino limite de truncado para visualizacion
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


/home/pablo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [10]:
# special characters
def caracteres_especiales(text):
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text) 
    return text


df['text']=df['text'].apply(caracteres_especiales)
df_test['text']=df_test['text'].apply(caracteres_especiales)

df.loc[5315:5320]

TypeError: expected string or bytes-like object

In [11]:
#Convierto el texto de a String
df['text'] = df['text'].astype("string")
df_test['text'] = df_test['text'].astype("string")

#convierto a minusculas
df['text'] = df['text'].str.lower()
df_test['text'] = df_test['text'].str.lower()

#Elimino las menciones, hashtag, urls.
df['text'] = df['text'].map(lambda text:  re.sub(r"(?:\@|&|#|https?\://)\S+", "", text))
df_test['text'] = df_test['text'].map(lambda text:  re.sub(r"(?:\@|&|#|https?\://)\S+", "", text))

#Elimino los simbolos @ sueltos
df['text'] = df['text'].map(lambda text: text.replace('@', ''))
df_test['text'] = df_test['text'].map(lambda text: text.replace('@', ''))

#Elimino numeros y palabras que contengan numeros.
df['text'] = df['text'].map(lambda text : re.sub(r'(?:\d)\S+','',text))
df_test['text'] = df_test['text'].map(lambda text : re.sub(r'(?:\d)\S+','',text))

#Elimino numeros y palabras que contengan numeros.
df['text'] = df['text'].map(lambda text : re.sub(r'(?:\d)','',text))
df_test['text'] = df_test['text'].map(lambda text : re.sub(r'(?:\d)','',text))

# Remuevo encerrado entre corchetes del texto del tweet
df['text'] = df['text'].map(lambda text: re.sub('([\[]).*?([\]])', '', text))
df_test['text'] = df_test['text'].map(lambda text: re.sub('([\[]).*?([\]])', '', text))

#quito signos de puntuacion
df['text'] = df['text'].map(lambda text: text.translate(str.maketrans('', '', string.punctuation)))
df_test['text'] = df_test['text'].map(lambda text: text.translate(str.maketrans('', '', string.punctuation)))

# eliminar signos de puntuacion puntuación
#df['text']=df['text'].map(lambda text: re.sub('[%s]' % re.escape(string.punctuation), ' ', text))


TypeError: expected string or bytes-like object

In [103]:
df.loc[5315:5320]

,id,keyword,location,text,target
5315,7589,outbreak,?? ?+254? ? \??å¡_??å¡_???å¡_?/??,families to sue over legionnaires more than families affected by the fatal outbreak of legionnaires disea,1
5316,7590,outbreak,NaN,an outbreak of legionnaires disease in new york has killed at least people now officials think theyve fo,1
5317,7591,outbreak,Dammam- KSA,families to sue over legionnaires more than families affected by the fatal outbreak of legionnaires disea,1
5318,7593,outbreak,"New York, NY",an outbreak of legionnaires disease in new york has killed at least people,1
5319,7594,outbreak,Pro-American and Anti-#Occupy,death confirmed in legionnaires outbreak in south bronx total cases reaches triple digits\n,1
5320,7595,pandemonium,NaN,tube strike absolute pandemonium,0


In [80]:
def convert_abbrev_in_text(text):
    t=[]
    words=text.split()
    t = [abbreviations[w.lower()] if w.lower() in abbreviations.keys() else w for w in words]
    return ' '.join(t) 


df['text']=df['text'].apply(convert_abbrev_in_text)
df_test['text']=df_test['text'].apply(convert_abbrev_in_text)

df.loc[5315:5320]

,id,keyword,location,text,target
5315,7589,outbreak,?? ?+254? ? \??å¡_??å¡_???å¡_?/??,families to sue over legionnaires more than families affected by the fatal outbreak of legionnaires disea,1
5316,7590,outbreak,NaN,an outbreak of legionnaires disease in new york has killed at least people now officials think theyve fo,1
5317,7591,outbreak,Dammam- KSA,families to sue over legionnaires more than families affected by the fatal outbreak of legionnaires disea,1
5318,7593,outbreak,"New York, NY",an outbreak of legionnaires disease in new york has killed at least people,1
5319,7594,outbreak,Pro-American and Anti-#Occupy,death confirmed in legionnaires outbreak in south bronx total cases reaches triple digits,1
5320,7595,pandemonium,NaN,tube strike absolute pandemonium,0


In [104]:
def remove_stop_words(text):
    

    text = tknzr.tokenize(text)
    
    # remove stopwords
    text = [w.lower() for w in text if not w in stop_words]
    
    # join back
    text = ' '.join(text)
    
    return text

df['text']=df['text'].apply(remove_stop_words)
df_test['text']=df_test['text'].apply(remove_stop_words)

df.loc[5315:5320]

,id,keyword,location,text,target
5315,7589,outbreak,?? ?+254? ? \??å¡_??å¡_???å¡_?/??,families sue legionnaires families affected fatal outbreak legionnaires disea,1
5316,7590,outbreak,NaN,outbreak legionnaires disease new york killed least people officials think theyve fo,1
5317,7591,outbreak,Dammam- KSA,families sue legionnaires families affected fatal outbreak legionnaires disea,1
5318,7593,outbreak,"New York, NY",outbreak legionnaires disease new york killed least people,1
5319,7594,outbreak,Pro-American and Anti-#Occupy,death confirmed legionnaires outbreak south bronx total cases reaches triple digits,1
5320,7595,pandemonium,NaN,tube strike absolute pandemonium,0


In [105]:
lista_sentencias = df['text'].tolist()
lista_sentencias_test = df_test['text'].tolist()

In [106]:
#inicio vectorizacion
from sklearn.feature_extraction.text import CountVectorizer

#primer vectorizer
#count_vectorizer = CountVectorizer(lista_sentencias, min_df=10 ,max_df=0.8, ngram_range=(1,2))
#train_vectors = count_vectorizer.fit_transform(lista_sentencias)

#list(count_vectorizer.vocabulary_)[:100]

# convierto todas las sentencias en array
#vectorizer.transform(lista_sentencias).toarray()
#fin primer vectorizer

count_vectorizer = CountVectorizer(min_df=10 ,max_df=0.8, ngram_range=(1, 2), stop_words = stop_words)
count_vectorizer.fit(df['text'])

train_vectors = count_vectorizer.transform(df['text'])
test_vectors = count_vectorizer.transform(df_test['text'])

In [107]:
#creo modelo
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

clf = LogisticRegression(C=1.0)
scores = model_selection.cross_val_score(clf, train_vectors, df['target'], cv=5, scoring="f1")
scores

array([0.57471264, 0.47579299, 0.57738573, 0.54561879, 0.68725869])

In [108]:
# Fitting a simple Logistic Regression on Counts
clf.fit(train_vectors, df['target'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [109]:
# Submission
sample_submission = pd.read_csv("dataset/sample_submission.csv")
sample_submission['target'] = clf.predict(test_vectors)
sample_submission.to_csv("submission1.csv", index=False)

In [14]:
#genero unigramas y bigramas para cada target
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(' ') if token != '' if token not in stop_words]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [' '.join(ngram) for ngram in ngrams]

N = 100

# Unigrams
disaster_unigrams = defaultdict(int)
nondisaster_unigrams = defaultdict(int)

for tweet in df[df['target']==1]['text']:
    for word in generate_ngrams(tweet):
        disaster_unigrams[word] += 1
        
for tweet in df[df['target']==0]['text']:
    for word in generate_ngrams(tweet):
        nondisaster_unigrams[word] += 1
        
df_disaster_unigrams = pd.DataFrame(sorted(disaster_unigrams.items(), key=lambda x: x[1])[::-1])
df_nondisaster_unigrams = pd.DataFrame(sorted(nondisaster_unigrams.items(), key=lambda x: x[1])[::-1])

disaster_unigrams

defaultdict(int,
            {'deeds': 1,
             'reason': 7,
             'may': 48,
             'allah': 4,
             'forgive': 1,
             'us': 80,
             'forest': 48,
             'fire': 176,
             'near': 47,
             'la': 21,
             'ronge': 1,
             'sask': 1,
             'canada': 6,
             'residents': 8,
             'asked': 1,
             'shelter': 7,
             'place': 14,
             'notified': 1,
             'officers': 4,
             'evacuation': 39,
             'orders': 9,
             'expected': 11,
             'people': 104,
             'receive': 2,
             'california': 107,
             'got': 29,
             'sent': 4,
             'photo': 17,
             'ruby': 1,
             'smoke': 12,
             'pours': 1,
             'school': 30,
             'update': 26,
             'hwy': 10,
             'closed': 17,
             'directions': 1,
             'due': 23,
             

In [ ]:
# Bigrams
disaster_bigrams = defaultdict(int)
nondisaster_bigrams = defaultdict(int)

for tweet in df[df['target']==1]['text']:
    for word in generate_ngrams(tweet, n_gram=2):
        disaster_bigrams[word] += 1
        
for tweet in df[df['target']==0]['text']:
    for word in generate_ngrams(tweet, n_gram=2):
        nondisaster_bigrams[word] += 1
        
df_disaster_bigrams = pd.DataFrame(sorted(disaster_bigrams.items(), key=lambda x: x[1])[::-1])
df_nondisaster_bigrams = pd.DataFrame(sorted(nondisaster_bigrams.items(), key=lambda x: x[1])[::-1])

In [7]:
#separo por grupos
dfreal = df[df['target']==1]
dffalso = df[df['target']==0]

dfreal['words']

0       [deeds, reason, earthquake, may, allah, forgive, us]                                                                            
1       [forest, fire, near, la, ronge, sask, canada]                                                                                   
2       [residents, asked, shelter, place, notified, officers, evacuation, shelter, place, orders, expected]                            
3       [people, receive, wildfires, evacuation, orders, california]                                                                    
4       [got, sent, photo, ruby, alaska, smoke, wildfires, pours, school]                                                               
                                      ...                                                                                               
7608    [two, giant, cranes, holding, bridge, collapse, nearby, homes]                                                                  
7609    [ariaahrary, thetawniest, control

In [22]:
lista = df['words'].tolist()
lista = sum(lista, [])
lista

['deeds',
 'reason',
 'earthquake',
 'may',
 'allah',
 'forgive',
 'us',
 'forest',
 'fire',
 'near',
 'la',
 'ronge',
 'sask',
 'canada',
 'residents',
 'asked',
 'shelter',
 'place',
 'notified',
 'officers',
 'evacuation',
 'shelter',
 'place',
 'orders',
 'expected',
 'people',
 'receive',
 'wildfires',
 'evacuation',
 'orders',
 'california',
 'got',
 'sent',
 'photo',
 'ruby',
 'alaska',
 'smoke',
 'wildfires',
 'pours',
 'school',
 'rockyfire',
 'update',
 'california',
 'hwy',
 'closed',
 'directions',
 'due',
 'lake',
 'county',
 'fire',
 'cafire',
 'wildfires',
 'flood',
 'disaster',
 'heavy',
 'rain',
 'causes',
 'flash',
 'flooding',
 'streets',
 'manitou',
 'colorado',
 'springs',
 'areas',
 'top',
 'hill',
 'see',
 'fire',
 'woods',
 'theres',
 'emergency',
 'evacuation',
 'happening',
 'building',
 'across',
 'street',
 'afraid',
 'tornado',
 'coming',
 'area',
 'three',
 'people',
 'died',
 'heat',
 'wave',
 'far',
 'haha',
 'south',
 'tampa',
 'getting',
 'flooded',
 '

In [24]:
listaset = set(lista)
listaset

{'bore',
 'listeria',
 'cafe',
 'platt',
 'ampor',
 'dublin',
 'dismisses',
 'saintrobinho86',
 'gum',
 'sneaks',
 'paranormal',
 'declared',
 'ruins',
 'shipwreck',
 'hopped',
 'preconditioning',
 'fuckfacewineisdumbcom',
 'prepared',
 'cityamp3others',
 'gregory',
 'atomicbomb',
 'contries',
 'ep016',
 'sweater',
 'aerospace',
 'walker',
 'biterelated',
 'cue',
 'bluedio',
 'thomassmonson',
 'wbcshirl2',
 'beth',
 'tonight',
 'pilloried',
 'metastatic',
 'sterlingknight',
 'ethical',
 'tedcruz2016',
 'withstand',
 'excessive',
 'blitzes',
 'butt',
 'bbcnews',
 '\x89û÷british',
 'avengers',
 'foodstand',
 'dye',
 'aesop',
 'humidity',
 'fx',
 'humboldt',
 'reason',
 '11st',
 'content',
 'throwback',
 'mean',
 'promoted',
 'firearm',
 'scandals',
 'christinalavv',
 'sundays',
 'bedroom',
 'obsolete',
 'transgender',
 'recently',
 'to\x89û',
 'bundled',
 'peale',
 'driveby',
 'postering',
 'bloggers',
 'motivator',
 'puppyshogun',
 'mental',
 'hug',
 'diarrhea',
 'homie',
 'pci',
 'grow

In [59]:
def creo_list_palabras(target):
    lista_palabras=[]
    
    for x in df[df['target']==target]['words']:
        for i in x:
            lista_palabras.append(i)
    return lista_palabras

# Dada una lista de palabras, devuelve un diccionario de
# pares de palabra-frecuencia.

def listaPalabrasDicFrec(listaPalabras):
    frecuenciaPalab = [listaPalabras.count(p) for p in listaPalabras]
    return dict(list(zip(listaPalabras,frecuenciaPalab)))

# Ordena un diccionario de pares palabra-frecuencia en
# orden de frecuencia descendente.

def ordenaDicFrec(dicfrec):
    aux = [(dicfrec[key], key) for key in dicfrec]
    aux.sort()
    aux.reverse()
    return aux

lista_palabras_reales = creo_list_palabras(0)
dic_frec_reales = ordenaDicFrec(listaPalabrasDicFrec(lista_palabras_reales))
dic_frec_reales

#lista_palabras_reales = creo_list_palabras(1)


[(253, 'like'),
 (168, 'new'),
 (112, 'body'),
 (99, 'via'),
 (91, 'people'),
 (90, 'video'),
 (89, 'love'),
 (85, 'know'),
 (84, 'back'),
 (83, 'us'),
 (83, 'time'),
 (83, 'got'),
 (82, 'see'),
 (81, 'full'),
 (81, 'emergency'),
 (81, 'cant'),
 (78, 'day'),
 (76, 'youtube'),
 (75, 'going'),
 (72, 'still'),
 (72, 'fire'),
 (67, 'want'),
 (67, 'good'),
 (65, 'think'),
 (64, '3'),
 (62, 'world'),
 (61, 'man'),
 (61, 'lol'),
 (60, 'rt'),
 (60, 'life'),
 (59, 'u'),
 (58, 'youre'),
 (58, 'first'),
 (56, 'news'),
 (56, 'last'),
 (55, 'way'),
 (55, 'really'),
 (55, 'need'),
 (55, 'burning'),
 (54, 'work'),
 (54, 'make'),
 (53, 'best'),
 (52, 'let'),
 (51, 'much'),
 (51, 'many'),
 (51, 'even'),
 (50, 'take'),
 (48, 'say'),
 (48, 'help'),
 (47, 'wreck'),
 (47, 'great'),
 (46, 'right'),
 (46, 'feel'),
 (46, 'content'),
 (45, 'hot'),
 (44, 'please'),
 (44, 'never'),
 (44, 'look'),
 (44, 'god'),
 (44, 'fear'),
 (44, 'every'),
 (43, 'could'),
 (43, '5'),
 (42, 'read'),
 (42, 'cross'),
 (42, 'bags')

In [14]:
#corrector de palabras
spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

df['text']=df['text'].apply(lambda x : correct_spellings(x))


'eunice joke iii she needs to chill and answer calmly its not like she s being attacked'